In [29]:
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
from enum import Enum
from typing import List
from openai import AzureOpenAI

In [30]:
def build_models():
    from openai import AzureOpenAI

    client = AzureOpenAI(
      api_key = "48ed1b0e5e8c4addac3156228ec6e61f",
      api_version = "2024-02-01",
      azure_endpoint = "https://blr-gpt-6.openai.azure.com/"
    )
    return client

In [31]:
ticket1 = """
I ordered a laptop from your store last week (Order #12345), but I received a tablet instead. 
This is unacceptable! I need the laptop for work urgently. Please resolve this immediately or I'll have to dispute the charge.
"""

ticket2 = """
Hello, I'm having trouble logging into my account. I've tried resetting my password, but I'm not receiving the reset email. 
Can you please help me regain access to my account? I've been a loyal customer for years and have several pending orders.
"""

In [32]:
client = build_models()


In [42]:
def classify_ticket_simple(ticket_text: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {"role": "system", "content": "Classify the following customer support ticket into a category."},
            {"role": "user", "content": ticket_text}
        ]
    )
    return response.choices[0].message.content

In [43]:
result = classify_ticket_simple(ticket1)
print(result)

Category: Order Issues


In [44]:
"""
Drawbacks of this approach:
1. No structured output, making it difficult to integrate into automated systems
2. No validation of the output, potentially leading to inconsistent categorizations
3. Limited information extracted, missing important details for prioritization
4. No confidence score, making it hard to flag uncertain classifications for human review
"""

'\nDrawbacks of this approach:\n1. No structured output, making it difficult to integrate into automated systems\n2. No validation of the output, potentially leading to inconsistent categorizations\n3. Limited information extracted, missing important details for prioritization\n4. No confidence score, making it hard to flag uncertain classifications for human review\n'

In [45]:
"""
Objective: Develop an AI-powered ticket classification system that:
- Accurately categorizes customer support tickets
- Assesses the urgency and sentiment of each ticket
- Extracts key information for quick resolution
- Provides confidence scores to flag uncertain cases for human review
Business impact:
- Reduce average response time by routing tickets to the right department
- Improve customer satisfaction by prioritizing urgent and negative sentiment tickets
- Increase efficiency by providing agents with key information upfront
- Optimize workforce allocation by automating routine classifications
"""

'\nObjective: Develop an AI-powered ticket classification system that:\n- Accurately categorizes customer support tickets\n- Assesses the urgency and sentiment of each ticket\n- Extracts key information for quick resolution\n- Provides confidence scores to flag uncertain cases for human review\nBusiness impact:\n- Reduce average response time by routing tickets to the right department\n- Improve customer satisfaction by prioritizing urgent and negative sentiment tickets\n- Increase efficiency by providing agents with key information upfront\n- Optimize workforce allocation by automating routine classifications\n'

In [46]:
client = instructor.patch(build_models())

In [47]:
class TicketCategory(str, Enum):
    ORDER_ISSUE = "order_issue"
    ACCOUNT_ACCESS = "account_access"
    PRODUCT_INQUIRY = "product_inquiry"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    OTHER = "other"

In [48]:
class CustomerSentiment(str, Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"
    SATISFIED = "satisfied"

In [49]:
class TicketUrgency(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

In [50]:
class TicketClassification(BaseModel):
    category: TicketCategory
    urgency: TicketUrgency
    sentiment: CustomerSentiment
    confidence: float = Field(ge=0, le=1, description="Confidence score for the classification")
    key_information: List[str] = Field(description="List of key points extracted from the ticket")
    suggested_action: str = Field(description="Brief suggestion for handling the ticket")


In [51]:
{
    'category': 'order_issue',
    'urgency': 'high',
    'sentiment': 'angry',
    'confidence': 0.9,
    'key_information': ['Customer received a tablet instead of a laptop', 'Order #12345 was placed last week', 'Laptop is needed urgently for work'],
    'suggested_action': 'Verify the order and ship the correct item to the customer as soon as possible'
}

ticket_classification = TicketClassification(
    category=TicketCategory.ORDER_ISSUE,
    urgency=TicketUrgency.HIGH,
    sentiment=CustomerSentiment.ANGRY,
    confidence=0.9,
    key_information=["Order #12345", "Received tablet instead of laptop"],
    suggested_action="Contact customer to arrange laptop delivery"
)

In [52]:
ticket_classification

TicketClassification(category=<TicketCategory.ORDER_ISSUE: 'order_issue'>, urgency=<TicketUrgency.HIGH: 'high'>, sentiment=<CustomerSentiment.ANGRY: 'angry'>, confidence=0.9, key_information=['Order #12345', 'Received tablet instead of laptop'], suggested_action='Contact customer to arrange laptop delivery')

In [53]:
def classify_ticket(ticket_text: str) -> TicketClassification:
    response = client.chat.completions.create(
        model="gpt-4o",
        response_model=TicketClassification,
        temperature=0,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": "Analyze the following customer support ticket and extract the requested information."
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

In [54]:
result1 = classify_ticket(ticket1)
result2 = classify_ticket(ticket2)

In [55]:
print(result1.model_dump_json(indent=2))
print("\n\n\n\n")
print(result2.model_dump_json(indent=2))

{
  "category": "order_issue",
  "urgency": "critical",
  "sentiment": "angry",
  "confidence": 0.95,
  "key_information": [
    "Customer ordered a laptop (Order #12345) but received a tablet instead",
    "Customer needs the laptop urgently for work",
    "Customer is considering disputing the charge"
  ],
  "suggested_action": "Apologize to the customer for the inconvenience, prioritize the correct shipment of the laptop, and provide a return label for the tablet. Consider offering a discount or expedited shipping as a goodwill gesture."
}





{
  "category": "account_access",
  "urgency": "high",
  "sentiment": "frustrated",
  "confidence": 0.9,
  "key_information": [
    "Customer is unable to log into their account.",
    "Password reset email not received.",
    "Customer has several pending orders.",
    "Customer has been a loyal customer for years."
  ],
  "suggested_action": "Verify the customer's email address and check the email server logs for any issues. Assist the cust